# How To Workbook
This workbook details how to use the functions for this module.

In [1]:
# Load required modules
import yaml
import pandas as pd

# Load functions from business day module
from business_day_finder import (
    combine_subdivision_holidays,
    generate_weekend_dates_list,
    weekend_and_holiday_dates,
    convert_to_date,
    find_business_day,
)


C:\Users\tyewf\AppData\Local\Temp\ipykernel_13564\3068659185.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Determine required countries and subdivisions
To determine the applicable business days, you will need to work out what set of public holidays to include. This can either come from the national holidays and/or subdivisions within a country.

In the 'country_subdivisions.xlsx' Excel file, there is a table with the complete list of countries and subdivisions that can be used. You must select one or more of these to use to determine the holiday dates. once selected, you will then need to create a dictionary of the country code and subdsivision to use ('' is used where it is the national holidays used). There is a yaml file 'subdivisions.yml' that already contains all of the dictionaries you might like to select from and use.

The process of selecting a country and its subdivision is detailed below:

In [2]:
# Read in 'country_subdivisions.xlsx'
subdivisions_table = pd.read_excel('country_subdivisions.xlsx', sheet_name='Sheet1')
subdivisions_table['Subdivisions'] = subdivisions_table['Subdivisions'].fillna('""')

# List the available countries
for country in sorted(subdivisions_table['Country'].unique()):
    print(country)

Albania
Algeria
American Samoa
Andorra
Angola
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Chad
Chile
China
Colombia
Costa Rica
Croatia
Cuba
Curacao
Cyprus
Czechia
Denmark
Djibouti
Dominican Republic
Ecuador
Egypt
El Salvador
Estonia
Eswatini
Ethiopia
Finland
France
Gabon
Georgia
Germany
Greece
Guam
Guatemala
Honduras
Hong Kong
Hungary
Iceland
India
Indonesia
Iran
Ireland
Isle of Man
Israel
Italy
Jamaica
Japan
Kazakhstan
Kenya
Kyrgyzstan
Laos
Latvia
Lesotho
Liechtenstein
Lithuania
Luxembourg
Madagascar
Malawi
Malaysia
Maldives
Malta
Marshall Islands (the)
Mexico
Moldova
Monaco
Montenegro
Morocco
Mozambique
Namibia
Netherlands
New Zealand
Nicaragua
Nigeria
North Macedonia
Northern Mariana Islands (the)
Norway
Pakistan
Panama
Papua New Guinea
Paraguay
Peru
Philippines
Poland
Portugal
Puerto Rico
Romania
Russia
San 

In [3]:
# Look for the available subdivisions to use:
country = 'Australia'

# Display the countrey code
code_value = subdivisions_table.loc[subdivisions_table['Country'] == country, 'Code'].values[0]
print(f'The Code for {country} is: {code_value}')

# Display teh available subdivisions
for subdivision in subdivisions_table[subdivisions_table['Country'] == country]['Subdivisions']:
    print(subdivision)

The Code for Australia is: AU
""
ACT
NSW
NT
QLD
SA
TAS
VIC
WA


In [4]:
# You can now make a dictionary with the countries and subdivisions
locations_dict = {
    'AU_NSW': {
        'country': 'AU',
        'subdivision': 'NSW',
    },
    'AU_VIC': {
        'country': 'AU',
        'subdivision': 'VIC',
    },
    'AU_NSW': {
        'country': 'AU',
        'subdivision': 'NSW',
    },
    'AU_QLD': {
        'country': 'AU',
        'subdivision': 'QLD',
    },
}
locations_dict

{'AU_NSW': {'country': 'AU', 'subdivision': 'NSW'},
 'AU_VIC': {'country': 'AU', 'subdivision': 'VIC'},
 'AU_QLD': {'country': 'AU', 'subdivision': 'QLD'}}

## Use predefined yaml
Alternatively, you can select the dictionaries from the pre-made yml dictionary

In [5]:
# Read the subdivisions YAML file
with open('subdivisions.yml', 'r') as file:
    yml_dict = yaml.safe_load(file)

yml_dict

{'Albania_': {'AL_': {'country': 'AL', 'subdivision': ''}},
 'Algeria_': {'DZ_': {'country': 'DZ', 'subdivision': ''}},
 'American Samoa_': {'AS_': {'country': 'AS', 'subdivision': ''}},
 'Andorra_': {'AD_': {'country': 'AD', 'subdivision': ''}},
 'Andorra_02': {'AD_02': {'country': 'AD', 'subdivision': '02'}},
 'Andorra_03': {'AD_03': {'country': 'AD', 'subdivision': '03'}},
 'Andorra_04': {'AD_04': {'country': 'AD', 'subdivision': '04'}},
 'Andorra_05': {'AD_05': {'country': 'AD', 'subdivision': '05'}},
 'Andorra_06': {'AD_06': {'country': 'AD', 'subdivision': '06'}},
 'Andorra_07': {'AD_07': {'country': 'AD', 'subdivision': '07'}},
 'Andorra_08': {'AD_08': {'country': 'AD', 'subdivision': '08'}},
 'Angola_': {'AO_': {'country': 'AO', 'subdivision': ''}},
 'Argentina_': {'AR_': {'country': 'AR', 'subdivision': ''}},
 'Armenia_': {'AM_': {'country': 'AM', 'subdivision': ''}},
 'Aruba_': {'AW_': {'country': 'AW', 'subdivision': ''}},
 'Australia_': {'AU_': {'country': 'AU', 'subdivisio

In [6]:
# Get all keys that start with the specified prefix
country = 'Australia'
matching_keys = {key: value for key, value in yml_dict.items() if key.startswith(country)}

# Print the matching keys
for key in matching_keys.keys():
    print(key)

Australia_
Australia_ACT
Australia_NSW
Australia_NT
Australia_QLD
Australia_SA
Australia_TAS
Australia_VIC
Australia_WA


In [7]:
# Create dict with subdivisions
locations_dict = {}
locations_dict.update(yml_dict['Australia_NSW'])
locations_dict.update(yml_dict['Australia_QLD'])

locations_dict

{'AU_NSW': {'country': 'AU', 'subdivision': 'NSW'},
 'AU_QLD': {'country': 'AU', 'subdivision': 'QLD'}}

# Create list of years
You will need a list of years that will be used to determine holiday dates

In [8]:
# One year:
years = [2025]

# Two years
years = [2024, 2025]

# Use functions to determine business days

## Generate list of holiday and weekend dates

In [9]:
weekends_and_holidays = weekend_and_holiday_dates(
    locations_dict=locations_dict,
    years=years
)
weekends_and_holidays

[datetime.date(2025, 4, 5),
 datetime.date(2024, 6, 22),
 datetime.date(2025, 6, 29),
 datetime.date(2024, 4, 21),
 datetime.date(2024, 11, 16),
 datetime.date(2024, 5, 25),
 datetime.date(2024, 6, 30),
 datetime.date(2025, 8, 2),
 datetime.date(2025, 10, 11),
 datetime.date(2024, 8, 3),
 datetime.date(2025, 2, 22),
 datetime.date(2024, 4, 6),
 datetime.date(2024, 2, 10),
 datetime.date(2024, 8, 24),
 datetime.date(2024, 11, 23),
 datetime.date(2025, 12, 13),
 datetime.date(2024, 1, 26),
 datetime.date(2025, 11, 22),
 datetime.date(2024, 3, 17),
 datetime.date(2024, 6, 16),
 datetime.date(2024, 12, 28),
 datetime.date(2025, 5, 18),
 datetime.date(2025, 3, 22),
 datetime.date(2024, 1, 28),
 datetime.date(2025, 4, 12),
 datetime.date(2025, 5, 24),
 datetime.date(2024, 9, 8),
 datetime.date(2024, 10, 26),
 datetime.date(2025, 11, 1),
 datetime.date(2025, 11, 16),
 datetime.date(2025, 8, 23),
 datetime.date(2025, 11, 30),
 datetime.date(2025, 4, 21),
 datetime.date(2024, 10, 7),
 datetime.

In [10]:
# You could also just get a list of holidays:
holiday_dates = combine_subdivision_holidays(
        locations_dict=locations_dict,
        years=years,
)

holiday_dates

[datetime.date(2024, 1, 1),
 datetime.date(2024, 3, 29),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 12, 25),
 datetime.date(2024, 1, 26),
 datetime.date(2024, 4, 25),
 datetime.date(2024, 12, 26),
 datetime.date(2024, 3, 30),
 datetime.date(2024, 3, 31),
 datetime.date(2024, 10, 7),
 datetime.date(2024, 6, 10),
 datetime.date(2024, 8, 5),
 datetime.date(2025, 1, 1),
 datetime.date(2025, 4, 18),
 datetime.date(2025, 4, 21),
 datetime.date(2025, 12, 25),
 datetime.date(2025, 1, 26),
 datetime.date(2025, 1, 27),
 datetime.date(2025, 4, 25),
 datetime.date(2025, 12, 26),
 datetime.date(2025, 4, 19),
 datetime.date(2025, 4, 20),
 datetime.date(2025, 10, 6),
 datetime.date(2025, 6, 9),
 datetime.date(2025, 8, 4),
 datetime.date(2024, 5, 6),
 datetime.date(2024, 8, 14),
 datetime.date(2025, 5, 5),
 datetime.date(2025, 8, 13)]

In [11]:
# You could also just get a list of weekends
weekend_dates = generate_weekend_dates_list(years=years)
weekend_dates

[datetime.date(2024, 1, 6),
 datetime.date(2024, 1, 7),
 datetime.date(2024, 1, 13),
 datetime.date(2024, 1, 14),
 datetime.date(2024, 1, 20),
 datetime.date(2024, 1, 21),
 datetime.date(2024, 1, 27),
 datetime.date(2024, 1, 28),
 datetime.date(2024, 2, 3),
 datetime.date(2024, 2, 4),
 datetime.date(2024, 2, 10),
 datetime.date(2024, 2, 11),
 datetime.date(2024, 2, 17),
 datetime.date(2024, 2, 18),
 datetime.date(2024, 2, 24),
 datetime.date(2024, 2, 25),
 datetime.date(2024, 3, 2),
 datetime.date(2024, 3, 3),
 datetime.date(2024, 3, 9),
 datetime.date(2024, 3, 10),
 datetime.date(2024, 3, 16),
 datetime.date(2024, 3, 17),
 datetime.date(2024, 3, 23),
 datetime.date(2024, 3, 24),
 datetime.date(2024, 3, 30),
 datetime.date(2024, 3, 31),
 datetime.date(2024, 4, 6),
 datetime.date(2024, 4, 7),
 datetime.date(2024, 4, 13),
 datetime.date(2024, 4, 14),
 datetime.date(2024, 4, 20),
 datetime.date(2024, 4, 21),
 datetime.date(2024, 4, 27),
 datetime.date(2024, 4, 28),
 datetime.date(2024, 5,

# Find relevant business day

## Find next business day

In [12]:
find_business_day(
    base_date= convert_to_date('2024-01-01'),
    weekends_and_holiday_dates=weekends_and_holidays,
    direction = 'next')

datetime.date(2024, 1, 2)

## Find previous business day

In [13]:
find_business_day(
    base_date= convert_to_date('2024-01-01'),
    weekends_and_holiday_dates=weekends_and_holidays,
    direction = 'previous')

datetime.date(2023, 12, 31)